In [ ]:
'''
# Naive Prediction Model
'''
import cbpro
import zmq
import sys
import json
import time
import os
import pickle
import pandas as pd
import numpy as np
import datetime as dt

# overload the on_message behavior of cbpro.WebsocketClient
class MyWebsocketClient(cbpro.WebsocketClient):
    def on_open(self):
        self.url = "wss://ws-feed.pro.coinbase.com/"
        self.products = symbol
        self.channels = ['ticker']
        self.should_print = False

    def on_message(self, msg):
        self.data = msg

    def on_close(self):
        print("-- Goodbye! --")

def logger_monitor(message, time=False, sep=False):
    # logger and monitor function
    with open(log_file, 'a') as f:
        t = str(dt.datetime.now())
        msg = ''
        if time:
            msg += ',' + t + ','
        if sep:
            msg += 3 * '='
        msg += ',' + message + ','
        # sends the message via the socket
        socket.send_string(msg)
        # writes the message to the log file
        f.write(msg)
        return

def logger_position(message):
    # logger and monitor function
    with open(fills_file, 'a') as f:
        # sends the message via the socket
        socket.send_string(message)
        # writes the message to the log file
        f.write(message)
        return

def report_positions(pos):
    '''Logs and sends position data'''
    out = str(pos)
    time.sleep(0.033) # waits for the order to be executed
    # get orders (will possibly make multiple HTTP requests)
    #get_orders_gen = auth_client.get_orders()
    get_fills = list(fills_gen)
    out += ',' + str(get_fills) + ','
    logger_position(out)
    return

# callback function - algo trading minimal working example
# https://en.wikipedia.org/wiki/Minimal_working_example

def trading_mwe(symbol, amount, position, bar, min_bars, twentyfour, df_accounts, df_fills):
    # Welcome message
    print('')
    print('*'*50)
    print('***      Welcome to Tenzin II Crypto Trader   ***')
    print('*'*50)
    print('')
    print('Trading: ', symbol)
    print('Amount per trade: ', amount)
    print('')
    print('Last 24 hrs:')
    print('')
    print('Open: .........', twentyfour['open'])
    print('Last: .........', twentyfour['last'])
    print('High: .........', twentyfour['high'])
    print('Low:    .......', twentyfour['low'])
    print('Volume:  ......', twentyfour['volume'])
    print('30 day Volume: ', twentyfour['volume_30day'])
    print('')
    print('Recent orders: ')
    #print(df_fills.loc[['product_id', 'fee', 'side', 'settled', 'usd_volume']])
    print('')
    print('Account Positions: ')
    print(df_accounts[['currency', 'balance']])
    print('')

    # global variables
    global wsClient, df, dataframe, hold, algorithm, log_file
    # intialize variables
    trading = 'n'   # default == not trading

    # ask to start trading
    trading = input('Start trading? [Y]/[n]:')

    if trading == 'Y':
        while wsClient.data:
            tick = wsClient.data
            dataframe = dataframe.append(tick, ignore_index=True)
            dataframe.index = pd.to_datetime(dataframe['time'], infer_datetime_format=True)
            # resampling of the tick data
            df = dataframe.resample(bar, label='right').last().ffill()

            if len(df) > min_bars:
                min_bars = len(df)
                logger_monitor('NUMBER OF TICKS: {} |'.format(len(dataframe))+\
                'NUMBER OF BARS: {}'.format(min_bars))
                # data processing and feature preparation
                df['price'] = df['price'].astype('float64')
                df['Returns'] = np.log(df['price']/df['price'].shift(1))
                df['Direction'] = np.where(df['Returns'] > 0, 1, -1)
                # picks relevant points
                features = df['Direction'].iloc[-(lags + 1): -1]
                # necessary reshaping
                features = features.values.reshape(1, -1)
                # generates the signal (+1 or -1)
                signal = df['Direction'].iloc[-1]          # naive prediction
                # signal = algorithm.predict(features)[0]
                # stores trade signal
                df['Position'] = position
                df['Signal'] = signal
    
                # logs and sends major financial information
                logger_monitor(str(df[['Returns', 'Direction', 'Position', 'Signal']].tail()))

                # trading logic
                if position in [1, -1] and signal == 1:
                    auth_client.place_market_order(product_id = symbol,
                                       side = 'buy', \
                                       funds = - position * amount)
                    position = 1
                    hold.append(position * amount)
                    report_positions('LONG')

                elif position in [0, 1] and signal == -1:
                    funds_held = np.sum(hold)
                    auth_client.place_market_order(product_id = symbol,\
                    side = 'sell', funds = str(funds_held))
                    position = -1
                    hold.append(-funds_held)
                    report_positions('SHORT')
                    
                else: # no trade
                    logger_monitor('no trade placed')

                logger_monitor(',****END OF CYCLE****,')
                #time.sleep(15.0)

            if len(df) > 288:
                # ends the trading session
                # long positions are held, open orders are closed
                logger_monitor(',ending trading session, max # ticks received,',\
                 True, False)
                # cancel orders
                report_positions(',CANCEL ORDERS,')
                auth_client.cancel_all(product_id=symbol)
                logger_monitor(',***CANCELING UNFILLED ORDERS***,')
                trading = 'n'


if __name__ == '__main__':
    # File path to save data to
    path = os.getcwd()                # for .ipynb implementation
    #path = os.path.dirname(__file__) # for .py implementation

    # log file to record trading
    t = str(time.time())
    log_file = 'trading_log-{}.csv'.format(t)
    fills_file = 'trading_fills-{}.csv'.format(t)

    # loads the persisted trading algorithm object
    algorithm = pd.read_pickle('algorithmBTC.pkl')

    # sets up the socket communication via ZeroMQ (here: "publisher")
    context = zmq.Context()
    socket = context.socket(zmq.PUB)

    # this binds the socket communication to all IP addresses of the machine
    # socket.bind('tcp://0.0.0.0:5555')
    # socket.bind('tcp://*:5555')
    socket.bind('tcp://*:5555')

    # Authentication credentials
    api_key = os.environ.get('CBPRO_SANDBOX_KEY')
    api_secret = os.environ.get('CBPRO_SANDBOX_SECRET')
    passphrase = os.environ.get('CBPRO_SANDBOX_PASSPHRASE')

    # sandbox authenticated client
    auth_client = cbpro.AuthenticatedClient(api_key, api_secret, passphrase, \
                                            api_url='https://api-public.sandbox.pro.coinbase.com')
    # live account authenticated client
    # uses a different set of API access credentials (api_key, api_secret, passphrase)
    # auth_client = cbpro.AuthenticatedCliet(api_key, api_secret, passphrase)

    # parameters for the trading algorithm
    # the trading algorithm runs silently for 500 ticks
    # use stratMonitoring.ipynb to monitor trading activity
    '''
    5 min: 300s, 10 min: 600s, 15 min: 900s, 30 min: 1800s, 45 min: 2700s
    1 hr: 3600s, 2hr: 7200s, 3hr: 10800s, 6hr: 21600s, 9hr: 32400s, 12hr: 43200s, 24hr: 86400s
    symbols: 'BTC-USD', 'ETH-BTC'
    '''

    symbol = 'ETH-BTC'
    bar = '300s'       # 15s is for testing; reset to trading frequency
    amount = 225        # amount to be traded in $USD
    position = 0        # beginning, neutral, position
    lags = 2            # number of lags for features data

    # minumum number of resampled bars required for the first predicted value (& first trade)
    min_bars = lags + 1

    # orders & fills generators to report positions:
    orders_gen = auth_client.get_orders()
    fills_gen = auth_client.get_fills(product_id=symbol)

    # Get stats for the last 24 hrs
    twentyfour = auth_client.get_product_24hr_stats(symbol)

    # Get filled orders
    all_fills = list(fills_gen)
    df_fills = pd.DataFrame(all_fills)
    #filepath = os.path.join(path, 'fills-{}.csv'.format(now))
    #df_fills.to_csv(filepath)

    # Get account positions
    accounts = auth_client.get_accounts()
    df_accounts = pd.DataFrame(accounts)
    #filepath = os.path.join(path, 'accounts-{}.csv'.format(now))
    #df_accounts.to_csv(filepath)

    # the main asynchronous loop using the callback function
    # Coinbase Pro web socket connection is rate-limited to 4 seconds per request per IP.

    wsClient = MyWebsocketClient()

    dataframe = pd.DataFrame() # dataframe for storing wsClient feed
    df = pd.DataFrame()        # dataframe for resampling wsClient feed
    hold = []         # amount held in position

    try:
        while True:
            # start trading
            wsClient.start()
            trading_mwe(symbol, amount, position, bar, min_bars, twentyfour, df_accounts, df_fills)
            # End session?
            tradeMore = input('Continue trading? [Y]/[n]:')
            if tradeMore == 'Y':
                trading_mwe(symbol, amount, position, bar, min_bars, twentyfour, df_accounts, df_fills)
            else:
                print('*** Tenzin trading session ended ***')
                wsClient.close()
                sys.exit(0)
    except KeyboardInterrupt:
        wsClient.close()

    if wsClient.error:
        print('Error - stopping program')
        wsClient.close()
        sys.exit(1)
    else:
        sys.exit(0)


**************************************************
***      Welcome to Tenzin II Crypto Trader   ***
**************************************************

Trading:  ETH-BTC
Amount per trade:  225

Last 24 hrs:

Open: ......... 0.03986
Last: ......... 0.04007
High: ......... 0.04123
Low:    ....... 0.0398
Volume:  ...... 2641.35807756
30 day Volume:  63241.78869906

Recent orders: 

Account Positions: 
  currency              balance
0      BAT     0.00000000000000
1      BTC     0.00000000000000
2      ETH     0.00000000000000
3      EUR     0.00000000000000
4      GBP     0.00000000000000
5     LINK     0.00000000000000
6      USD  1277.46760810417265
7     USDC     0.00000000000000



Start trading? [Y]/[n]: Y


In [23]:
import numpy as np
import pandas as pd

hold = []
position = 1
amount = 225
hold.append(position * amount)

funds_held = np.sum(hold)
hold.append(-funds_held)
print(funds_held)
print(hold)

-225
[-225, 225]
